Load deps


In [5]:
import numpy as np
import tensorflow as tf
import sys
import os

sys.path.insert(0, f"{os.getcwd()}/../../../../ML/preprocess/encoding/letter")
from encoding import encode


Get data


In [7]:
x_train, y_train = encode()


In [8]:
print(x_train[100][:][:])
print(np.shape(x_train[0][:][:]))
print(y_train[100][:][:])
print(np.shape(y_train[0][:][:]))

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [9]:
NB_SAMPLES = 100000

# Take rand portion of the data train samples
np.random.seed(2022)
rand_indices = np.random.permutation(len(x_train))
rand_train_indices = rand_indices[:NB_SAMPLES]
rand_test_indices = rand_indices[NB_SAMPLES : NB_SAMPLES + 300]
xs = x_train[rand_train_indices]
ys = y_train[rand_train_indices]
x_test = x_train[rand_test_indices]
y_test = y_train[rand_test_indices]

Build model


In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Bidirectional,
    LSTM,
    Flatten,
    Dropout,
    Dense,
)

TIMESTEPS = 36  # max word length
DATA_DIM = 27  # number of letters + 1 (catch-all char)
DROPOUT = 0.8

input_shape = (TIMESTEPS, DATA_DIM)
forward_layer = LSTM(TIMESTEPS, return_sequences=True)
backward_layer = LSTM(TIMESTEPS, return_sequences=True, go_backwards=True)

model = Sequential()
model.add(
    Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=input_shape)
)
model.add(Flatten())
model.add(Dense(570, activation="gelu"))
model.add(Dropout(DROPOUT))
model.add(Dense(15, activation="gelu"))
model.add(Dense(1))
model.compile(
    optimizer="Adam", loss="mean_absolute_error", metrics=["mean_absolute_error"],
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 36, 72)           18432     
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 2592)              0         
                                                                 
 dense (Dense)               (None, 570)               1478010   
                                                                 
 dropout (Dropout)           (None, 570)               0         
                                                                 
 dense_1 (Dense)             (None, 15)                8565      
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                        

In [39]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

BATCH_SIZE = 30
EPOCHS = 100

# Add early stopping and tensorboard callbacks
early_stopping_callback = EarlyStopping(monitor="loss", patience=10)
tensorboard_callback = TensorBoard(".logdir")
callbacks = [tensorboard_callback, early_stopping_callback]

model.fit(
    x=xs,
    y=ys,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    shuffle=True,
    epochs=EPOCHS,
    callbacks=[callbacks],
)

model.save("../../../../models/blstm_on_natural_data_model.h5")

Epoch 1/100
3000/3000 [==============================] - 118s 38ms/step - loss: 0.0914 - mean_absolute_error: 0.0914 - val_loss: 0.0651 - val_mean_absolute_error: 0.0651
Epoch 2/100
3000/3000 [==============================] - 110s 37ms/step - loss: 0.0774 - mean_absolute_error: 0.0774 - val_loss: 0.0627 - val_mean_absolute_error: 0.0627
Epoch 3/100
3000/3000 [==============================] - 112s 37ms/step - loss: 0.0696 - mean_absolute_error: 0.0696 - val_loss: 0.0618 - val_mean_absolute_error: 0.0618
Epoch 4/100
3000/3000 [==============================] - 113s 38ms/step - loss: 0.0608 - mean_absolute_error: 0.0608 - val_loss: 0.0448 - val_mean_absolute_error: 0.0448
Epoch 5/100
3000/3000 [==============================] - 120s 40ms/step - loss: 0.0560 - mean_absolute_error: 0.0560 - val_loss: 0.0466 - val_mean_absolute_error: 0.0466
Epoch 6/100
3000/3000 [==============================] - 116s 39ms/step - loss: 0.0517 - mean_absolute_error: 0.0517 - val_loss: 0.0394 - val_mean_abs

Test model

In [11]:
from tensorflow.keras.models import load_model

print("Evaluate on test data")
loaded_model = load_model("../../../../models/blstm_on_natural_data_model.h5")
results = loaded_model.evaluate(x_test, y_test, batch_size=30)
print("test loss, test acc:", results)

Evaluate on test data
10/10 [==============================] - 1s 13ms/step - loss: 0.0228 - mean_absolute_error: 0.0228
test loss, test acc: [0.022822052240371704, 0.022822052240371704]


Try this model with a simulated user_test_word. Change user_test_word to any word you like. It's specially revealing if you use English words that are not in the syllableCountDict (such as pogchamp).

In [12]:
from tensorflow.keras.models import load_model
from encoding import encode_word


def predict(word):
    encoded_word = encode_word(word)
    loaded_model = load_model("../../../../models/blstm_on_natural_data_model.h5")
    out_prediction = loaded_model.predict(np.array([encoded_word]))
    return out_prediction


user_test_word = "pogchamp"  # Change this to simulate arbitrary user input.
print(predict(user_test_word))



[[1.9606566]]
